In [ ]:
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications import VGG16
from keras.models import Model
import numpy as np
import cv2
import os

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow 

In [ ]:
model = VGG16()
model.layers.pop()
model = Model(model.inputs,model.layers[-1].output)

In [ ]:
model.output

In [ ]:
def extract_vgg16_features(video_input_file_path):
    count = 0
    #print('Extracting frames from video: ', video_input_file_path)
    vidcap = cv2.VideoCapture(video_input_file_path)
    success, image = vidcap.read()
    features = []
    success = True
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC, (count * 1000))  # added this line
        success, image = vidcap.read()
        # print('Read a new frame: ', success)
        if success:
            img = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
            input = img_to_array(img)
            input = np.expand_dims(input, axis=0)
            input = preprocess_input(input)
            feature = model.predict(input).ravel()
            features.append(feature)
            count = count + 1
    unscaled_features = np.array(features)
    #np.save(feature_output_file_path, unscaled_features)
    return unscaled_features

In [ ]:
y_samples = []
x_samples = []

In [ ]:
input_data_dir_path = 'UCF-101'

In [ ]:
output_feature_data_dir_path = 'vgg16-features'

In [ ]:
dir_count = 0
for f in os.listdir(input_data_dir_path):
    file_path = input_data_dir_path + os.path.sep + f
    output_dir_name = f
    output_dir_path = output_feature_data_dir_path + os.path.sep + output_dir_name
    dir_count += 1
    for ff in os.listdir(file_path):
        video_file_path = file_path + os.path.sep + ff
        #output_feature_file_path = output_dir_path + os.path.sep + ff.split('.')[0] + '.npy'
        x = extract_vgg16_features(video_file_path)
        y = f
        #print(x.shape)
        y_samples.append(y)
        x_samples.append(x)
    print(dir_count)

In [ ]:
x = np.array(x_samples)
y = np.array(y_samples)

In [ ]:
np.save('xf.npy', x)

In [ ]:
np.save('yf.npy', y)

### save here before continue because this steps took a lot of time

In [1]:

from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [2]:
x_samples = np.load('xf.npy',allow_pickle=True)

In [3]:
y_samples = np.load('yf.npy', allow_pickle=True)

In [4]:
max_frames = 0

In [5]:
frames_list = []
for x in x_samples:
    frames = x.shape[0]
    frames_list.append(frames)
    max_frames = max(frames, max_frames)
expected_frames = int(np.mean(frames_list))

In [6]:
len(x_samples)

13320

In [21]:
xt = np.zeros((1,7,4096))

In [22]:
for i in range(len(x_samples)):
    #print(i)
    x = x_samples[i]
    frames = x.shape[0]
    if frames >= expected_frames:
        x = x[0:expected_frames, :]
        xt =  np.concatenate((xt,x.reshape((1,x.shape[0],x.shape[1]))),axis=0)
    elif frames < expected_frames:
        temp = np.zeros(shape=(expected_frames, x.shape[1]))
        temp[0:frames, :] = x
        xt =  np.concatenate((xt,temp.reshape((1,temp.shape[0],temp.shape[1]))),axis=0)
    if i % 1000==0:
        print(i, xt.shape)

0 (2, 7, 4096)
1000 (1002, 7, 4096)
2000 (2002, 7, 4096)
3000 (3002, 7, 4096)
4000 (4002, 7, 4096)
5000 (5002, 7, 4096)
6000 (6002, 7, 4096)
7000 (7002, 7, 4096)
8000 (8002, 7, 4096)
9000 (9002, 7, 4096)
10000 (10002, 7, 4096)
11000 (11002, 7, 4096)
12000 (12002, 7, 4096)
13000 (13002, 7, 4096)


In [23]:
expected_frames

7

In [24]:
labels = dict()
for y in y_samples:
    if y not in labels:
        labels[y] = len(labels)
print(labels)
for i in range(len(y_samples)):
    y_samples[i] = labels[y_samples[i]]

{'Archery': 0, 'Knitting': 1, 'PlayingDaf': 2, 'CricketBowling': 3, 'HammerThrow': 4, 'GolfSwing': 5, 'Skiing': 6, 'JugglingBalls': 7, 'SalsaSpin': 8, 'BenchPress': 9, 'Fencing': 10, 'WritingOnBoard': 11, 'PommelHorse': 12, 'IceDancing': 13, 'Rowing': 14, 'CliffDiving': 15, 'PullUps': 16, 'Diving': 17, 'VolleyballSpiking': 18, 'BoxingPunchingBag': 19, 'PlayingSitar': 20, 'Billiards': 21, 'FieldHockeyPenalty': 22, 'PlayingCello': 23, 'PlayingTabla': 24, 'BreastStroke': 25, 'SumoWrestling': 26, 'Biking': 27, 'HorseRiding': 28, 'ApplyEyeMakeup': 29, 'Bowling': 30, 'TennisSwing': 31, 'Hammering': 32, 'BasketballDunk': 33, 'PushUps': 34, 'ApplyLipstick': 35, 'TrampolineJumping': 36, 'PoleVault': 37, 'ThrowDiscus': 38, 'BrushingTeeth': 39, 'TaiChi': 40, 'Surfing': 41, 'Rafting': 42, 'PlayingPiano': 43, 'BalanceBeam': 44, 'HandstandPushups': 45, 'WalkingWithDog': 46, 'Mixing': 47, 'MilitaryParade': 48, 'SoccerJuggling': 49, 'JumpRope': 50, 'Kayaking': 51, 'BaseballPitch': 52, 'LongJump': 53, 

In [25]:
xt.shape

(13321, 7, 4096)

In [26]:
#x = np.array(x_samples)
y = np.array(y_samples)
y = to_categorical(y,num_classes=101)

In [29]:
xt = xt[1:]

In [30]:
print(xt.shape, y.shape)

(13320, 7, 4096) (13320, 101)


In [31]:
np.save('xfe.npy', xt)
np.save('yfe.npy', y)

In [32]:
np.save('labels.npy', labels)